# Example

This demonstrates how to extract the rules and underlying model and convert it to run on the SQL Engine rather than via Python. This should hopefully make the code mor portable across databases and allow it to scale for deployment.

In [53]:
import pandas as pd
import numpy as np
from sklearn import datasets
import sqlite3
from skrules import SkopeRules
from sklearn_predict.feature_engineer import SkopeRulesSQL

In [59]:
conn = sqlite3.connect(":memory:")
iris = datasets.load_iris()
feat_names = ['_'.join(x.split()[:2]) for x in iris['feature_names']]
tbl_name = "iris"

pd.DataFrame(data= np.c_[iris['data'], iris['target']],
             columns= feat_names + ['target']).to_sql(tbl_name, conn, index=False, if_exists="replace")

df = pd.read_sql("select * from iris", conn)
model_matrix = df[[x for x in df.columns if x not in [ "target"]]]
model_matrix.head()

,sepal_length,sepal_width,petal_length,petal_width
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [61]:
clf = SkopeRules(max_depth_duplication=2,
                n_estimators=30,
                precision_min=0.3,
                recall_min=0.1,
                feature_names=model_matrix.columns)
clf.fit(model_matrix, df['target'])
export = SkopeRulesSQL(clf, feat_names)

/home/chapman/anaconda3/envs/topdown/lib/python3.8/site-packages/skrules/skope_rules.py:214: UserWarning: Found labels {0.0, 1.0, 2.0}. This method assumes target class to be labeled as 1 and normal data to be labeled as 0. Any label different from 0 will be considered as being from the target class.
  warn("Found labels %s. This method assumes target class to be"


In [64]:
query_sql = ["{} end as rule{}".format(x, idx) for idx, x in enumerate(export.export())]
query_sql = "select {}, * from {}".format(', '.join(query_sql), tbl_name)
print(query_sql)

select CASE WHEN `petal_width` > 0.7000000029802322 THEN 1 ELSE 0 end as rule0, CASE WHEN `petal_length` > 2.599999964237213 THEN 1 ELSE 0 end as rule1, * from iris


In [67]:
tbl_feat = "iris_feat"

c = conn.cursor()
c.execute(f"create table {tbl_feat} as {query_sql}")
conn.commit()
pd.read_sql(f"select * from {tbl_feat}", conn)

,rule0,rule1,sepal_length,sepal_width,petal_length,petal_width,target
0,0,0,5.1,3.5,1.4,0.2,0.0
1,0,0,4.9,3.0,1.4,0.2,0.0
2,0,0,4.7,3.2,1.3,0.2,0.0
3,0,0,4.6,3.1,1.5,0.2,0.0
4,0,0,5.0,3.6,1.4,0.2,0.0
...,...,...,...,...,...,...,...
145,1,1,6.7,3.0,5.2,2.3,2.0
146,1,1,6.3,2.5,5.0,1.9,2.0
147,1,1,6.5,3.0,5.2,2.0,2.0
148,1,1,6.2,3.4,5.4,2.3,2.0
